In [59]:
# All imports
import RobotDynamics
import RigidBodyDynamics
import Random
Random.seed!(1);
using LinearAlgebra
# using StaticArrays

In [60]:
# Defining Mechanism and MechanismState for RigidBodyDynamics
Arthur = RigidBodyDynamics.URDF.parse_urdf("/home/amkyu/catkin_workspace/src/ros_kortex/kortex_description/arms/gen3/7dof/urdf/GEN3_URDF_V12.urdf")
ArthurState = RigidBodyDynamics.MechanismState(Arthur)

MechanismState{Float64, Float64, Float64, …}(…)

In [69]:
# TODO: Change later!!!
# Randomizing joint state (position and velocity), and randomizing input torques
Random.rand!(ArthurState)
torques = Random.rand!(zeros(7))/1000
# External Force measurement
# Newtons [Fx, Fy, Fz, Mx, My, Mz]
# Only Mz is non-zero since all forces are applied directly to the reamer hemisphere
Fe = zeros(6) 
Fe[1:3] = Random.rand(3)
p = RigidBodyDynamics.path(Arthur, RigidBodyDynamics.root_body(Arthur), RigidBodyDynamics.bodies(Arthur)[end])
J_data = RigidBodyDynamics.geometric_jacobian(ArthurState, p)
J = [J_data.linear; J_data.angular]
q̇ = RigidBodyDynamics.velocity(ArthurState)
ẋ = J*q̇
Be = zeros(6, 6)
for k = 1:3
    Be[k,k] = norm(Fe) / norm(ẋ)
end

# Combine joint position and velocities into a collective state vector
s = [RigidBodyDynamics.configuration(ArthurState)[:]; RigidBodyDynamics.velocity(ArthurState)[:]; diag(Be)]


# See state and torques
print(q̇)
print("\n\n")
print(J)
print("\n\n")
print(ẋ)
print("\n\n")
print(Be)
print("\n\n")
print(s)
print("\n\n")
print(torques)

[0.10283157795639708, 0.6704205825614975, 0.7554152239466787, 0.6490562651461806, 0.6604003260599425, 0.9062106456676113, 0.27541564882559744]

[-1.1491989486252564e-6 -0.2623895342574835 0.04781026847113355 -0.5868413906024228 -0.3103839579883502 -0.42560534635440916 0.5512066015398971; -4.322004469883372e-19 0.11076316920697019 0.08484310577283624 0.3459158024462122 -0.5515387929749732 0.6494741237986119 -0.029141751582740474; 0.0 3.667859762314894e-7 0.0039923389947176015 -0.14025181822883132 -0.03774641112150598 0.11189601597424628 -0.05022469789688772; -2.7628999999254435e-18 0.38890193999522765 0.3129849338346425 0.49935486041262506 -0.6881189315162038 0.7138114029649424 -0.10464329136319034; 7.346410206643587e-6 0.9212791547402597 -0.1321144630690906 0.865246914805559 0.42737668002561724 0.5437984668542785 -0.5402557221721582; -0.9999999999730151 1.0441299689179076e-5 -0.9405244280934388 0.04463742602064711 -0.5863800043139935 -0.44131225730105883 -0.8349691828043223]

[-0.95954

In [82]:
## Dynamics Function
# model is the parsed URDF of the robotic arm
# x is joint position, velocity, and the B_External constants (6 for each DOF in Cartesian Space)
# u is joint torques

function RobotDynamics.dynamics(model::RigidBodyDynamics.Mechanism, x, u)
    dynamicsResult = RigidBodyDynamics.DynamicsResult(model)
    mechanismState = RigidBodyDynamics.MechanismState(model)
    M = RigidBodyDynamics.mass_matrix(mechanismState)
    num_q = RigidBodyDynamics.num_positions(model)
    q = x[1:num_q]
    qd = x[num_q+1:2*num_q]
    Be = diagm(x[end-5:end])
    p = RigidBodyDynamics.path(Arthur, RigidBodyDynamics.root_body(Arthur), RigidBodyDynamics.bodies(Arthur)[end])
    J_data = RigidBodyDynamics.geometric_jacobian(ArthurState, p)
    J = [J_data.linear; J_data.angular]
    q̇ = RigidBodyDynamics.velocity(ArthurState)
    ẋ = J*q̇
    τ_ext = transpose(J)*Be*ẋ
    RigidBodyDynamics.set_configuration!(mechanismState, q)
    RigidBodyDynamics.set_velocity!(mechanismState, qd)
    RigidBodyDynamics.dynamics!(dynamicsResult, mechanismState, u)
    qdd = M\((M * dynamicsResult.v̇) - τ_ext)
    return [qd; qdd; 0; 0; 0; 0; 0; 0]
end

RobotDynamics.state_dim(::Cartpole) = 4
RobotDynamics.control_dim(::Cartpole) = 1

In [83]:
print(RobotDynamics.dynamics(Arthur, s, torques))

[0.10283157795639708, 0.6704205825614975, 0.7554152239466787, 0.6490562651461806, 0.6604003260599425, 0.9062106456676113, 0.27541564882559744, 0.16313006186371126, -10.06332677956533, -3.3746019310993063, 47.62136703656684, -252.50596682139314, -108.66567582087993, 608.0082841480627, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]